<a href="https://colab.research.google.com/github/Partha-SUST16/LLM_JailBreak_Low_Resource/blob/main/python_scripts/Jailbreaking_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
import time
from groq import Groq

class LLMClient:
    def __init__(self, api_key):
        self.client = Groq(api_key=api_key)
        self.models = {
            "llama3-70b-8192": {
                "requests_per_minute": 30,
                "tokens_per_minute": 5000,
                "requests_per_day": 14400,
                "tokens_per_day": 500000,
                "request_count": 0,
                "token_count": 0,
                "last_request_time": time.time(),
            },
            "mixtral-8x7b-32768": {
                "requests_per_minute": 30,
                "tokens_per_minute": 4000,
                "requests_per_day": 14400,
                "tokens_per_day": 500000,
                "request_count": 0,
                "token_count": 0,
                "last_request_time": time.time(),
            },
            # Add more models and their rate limits here
        }

    def _estimate_tokens(self, text):
        # Approximate token count (1 token ~= 4 characters)
        return max(1, len(text) // 4)

    def _check_rate_limits(self, model, prompt_tokens, response_tokens):
        model_data = self.models[model]
        current_time = time.time()
        elapsed_time = current_time - model_data["last_request_time"]

        # Reset counters if a minute has passed
        if elapsed_time >= 60:
            model_data["request_count"] = 0
            model_data["token_count"] = 0
            model_data["last_request_time"] = current_time

        # Check per-minute limits
        if model_data["request_count"] >= model_data["requests_per_minute"]:
            time.sleep(60 - elapsed_time)  # Wait until the next minute
            model_data["request_count"] = 0
            model_data["token_count"] = 0
            model_data["last_request_time"] = time.time()

        if model_data["token_count"] + prompt_tokens + response_tokens > model_data["tokens_per_minute"]:
            time.sleep(60 - elapsed_time)  # Wait until the next minute
            model_data["token_count"] = 0
            model_data["last_request_time"] = time.time()

        # Check daily limits
        if model_data["request_count"] >= model_data["requests_per_day"]:
            raise Exception(f"Daily request limit reached for model {model}.")
        if model_data["token_count"] + prompt_tokens + response_tokens > model_data["tokens_per_day"]:
            raise Exception(f"Daily token limit reached for model {model}.")

    def generate_response(self, prompt, model):
        # Ensure the response is in English by appending an instruction
        english_prompt = f"{prompt}\n\nRespond in English regarless of the promts language."

        # Estimate tokens for the prompt
        prompt_tokens = self._estimate_tokens(english_prompt)
        # Assume a maximum response length of 100 tokens for estimation
        response_tokens = 100

        # Check and enforce rate limits for the specific model
        self._check_rate_limits(model, prompt_tokens, response_tokens)

        try:
            # Send the prompt to the model using Groq
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": english_prompt,
                    }
                ],
                model=model,
            )

            # Get the response
            response = chat_completion.choices[0].message.content

            # Update counters for the specific model
            self.models[model]["request_count"] += 1
            self.models[model]["token_count"] += prompt_tokens + self._estimate_tokens(response)

            return response
        except Exception as e:
            print(f"Error generating response for model {model}: {e}")
            return None

In [ ]:
def process_csv(input_file, output_file, llm_client, model):
    # Open the input CSV file
    with open(input_file, mode="r", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)  # Read all rows into a list

    # Open the output CSV file in append mode
    with open(output_file, mode="a", encoding="utf-8", newline="") as outfile:
        fieldnames = ["id", "translated_prompt", "cosine_similarity","response"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the header only if the file is empty
        if outfile.tell() == 0:
            writer.writeheader()

        # Process each row
        for row in rows:
            id = row["id"]
            translated_prompt = row["translated_prompt"]

            # Generate the response using the specified model
            response = llm_client.generate_response(translated_prompt, model)

            # Write the result to the output CSV immediately
            writer.writerow({
                "id": id,
                "translated_prompt": translated_prompt,
                "cosine_similarity": row["cosine_similarity"],
                "response": response,
            })

            print(f"Processed row {id} with model {model}")  # Log progress

    print(f"Responses saved to {output_file}")

In [ ]:
def process_mixed_prompt_csv(input_file, output_file, llm_client, model):
    # Open the input CSV file
    with open(input_file, mode="r", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)  # Read all rows into a list

    # Open the output CSV file in append mode
    with open(output_file, mode="a", encoding="utf-8", newline="") as outfile:
        fieldnames = ["id", "translated_prompt", "response"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the header only if the file is empty
        if outfile.tell() == 0:
            writer.writeheader()

        # Process each row
        for row in rows:
            id = row["id"]
            translated_prompt = row["Translated Prompt"]

            # Generate the response using the specified model
            response = llm_client.generate_response(translated_prompt, model)

            # Write the result to the output CSV immediately
            writer.writerow({
                "id": id,
                "translated_prompt": translated_prompt,
                "response": response,
            })

            print(f"Processed row {id} with model {model}")  # Log progress

    print(f"Responses saved to {output_file}")

In [ ]:
def process_file_with_model(file_path, output_dir, api_key, model):
    # Initialize the LLMClient
    llm_client = LLMClient(api_key)

    # Generate the output file path based on the input file name and model
    file_name = os.path.basename(file_path)
    output_file = os.path.join(output_dir, f"{model}_{file_name}")

    # Call the process_csv function
    process_mixed_prompt_csv(file_path, output_file, llm_client, model)
    #process_csv(file_path, output_file, llm_client, model)

In [ ]:
import os
import concurrent.futures
from groq import Groq

def process_all_files_in_parallel(directory, output_dir, api_key, models):
    # Get a list of all files in the directory
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # Use ThreadPoolExecutor to run tasks in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for file_path in files:
            for model in models:
               if file_path.endswith('bn_common_prompts.csv') and model == 'llama3-70b-8192':
                  continue
                # Submit the task to the executor
               futures.append(executor.submit(process_file_with_model, file_path, output_dir, api_key, model))

        # Wait for all tasks to complete
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # This will raise any exceptions that occurred during execution
            except Exception as e:
                print(f"Error processing file: {e}")



# Example usage
directory = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons"  # Replace with the path to your input files directory
output_dir = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response"  # Replace with the path to your output files directory
models = ["llama3-70b-8192", "mixtral-8x7b-32768"]  # List of models to use

process_all_files_in_parallel(directory, output_dir, token, models)

Streaming output truncated to the last 5000 lines.
Error generating response for model mixtral-8x7b-32768: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01jh1brkxfevqs9ybzn0pp3nqe` service tier `on_demand` on : Limit 500000, Used 499983, Requested 848. Please try again in 2m23.4596s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Processed row 1675 with model mixtral-8x7b-32768
Error generating response for model mixtral-8x7b-32768: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01jh1brkxfevqs9ybzn0pp3nqe` service tier `on_demand` on : Limit 500000, Used 499983, Requested 510. Please try again in 1m25.0222s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Processed row 1476 with model mixtral-8x7b-32768
Error generating re

In [ ]:
import os
import concurrent.futures
import csv

def get_failed_rows(output_file):
    """Read the output file and return rows where the response is empty or None."""
    failed_rows = []
    with open(output_file, mode="r", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if not row.get("response"):  # Check if the response is empty or None
                failed_rows.append(row)
    return failed_rows

def retry_failed_rows(output_file, llm_client, model):
    """Retry generating responses for rows without a response and update the file immediately."""
    # Create a temporary file to write updated rows
    temp_file = output_file + ".tmp"
    cache = {}
    if os.path.exists(temp_file):
        with open(temp_file, mode="r", encoding="utf-8") as cache_file:
            cache_reader = csv.DictReader(cache_file)
            for row in cache_reader:
                cache[row["id"]] = row

    with open(output_file, mode="r", encoding="utf-8") as infile, open(temp_file, mode="w", encoding="utf-8", newline="") as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        cnt = 0
        for row in reader:
          try:
            if row["id"] in cache and cache[row["id"]].get("response"):
                    row["response"] = cache[row["id"]]["response"]  # Use cached response
                    print(f"Using cached response for row {row['id']}")
            elif not row.get("response"):  # Check if the response is empty or None
                translated_prompt = row["translated_prompt"]
                response = llm_client.generate_response(translated_prompt, model)
                if response:
                    row["response"] = response  # Update the response
                    print(f"Updated response for row {row['id']}")

            # Write the row to the temporary file
            writer.writerow(row)
          except Exception as e:
            print(f"Error generating response for row {row['id']}: {e}")
          print(f"Processed row {cnt}")  # Log progress
          cnt += 1
    # Replace the original file with the updated file
    os.replace(temp_file, output_file)
    print(f"Retry completed for {output_file}.")

In [ ]:
directory = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons"  # Replace with the path to your input files directory
output_dir = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response"  # Replace with the path to your output files directory
models = ["llama3-70b-8192", "mixtral-8x7b-32768"]  # List of models to use
api_key = ''
# retry_failed_responses(directory, output_dir, api_key, models)

Error generating response for model mixtral-8x7b-32768: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01jm97cvvdevrs816y5w62w1c7` service tier `on_demand` on tokens per minute (TPM): Limit 5000, Used 4188, Requested 1048. Please try again in 2.831s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
['id', 'translated_prompt', 'cosine_similarity', 'response']
Retry completed for /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_bn_common_prompts.csv.
['id', 'translated_prompt', 'response']
Retry completed for /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_bn_common_prompts.csv.
['id', 'translated_prompt', 'cosine_similarity', 'response']
Updated response for row 39 in /content/drive/MyDrive/CPSC 538P Project (Computer

KeyboardInterrupt: 

In [ ]:
api_key = ""
output_path_bengla = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_bn_common_prompts.csv'
llm_client = LLMClient(api_key)
failed_rows = get_failed_rows(output_path_bengla)
retry_failed_rows(output_path_bengla, llm_client, 'llama3-70b-8192')

['id', 'translated_prompt', 'cosine_similarity', 'response']
Retry completed for /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_bn_common_prompts.csv.


In [ ]:
output_path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_dv_common_prompts.csv'
llm_client = LLMClient('')
retry_failed_rows(output_path, llm_client, 'llama3-70b-8192')

Updated response for row 1803 in /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_dv_common_prompts.csv
Retry completed for /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_dv_common_prompts.csv.


In [ ]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

token = ''
llm_client = LLMClient(token)
output_path_dv = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_dv_common_prompts.csv'
retry_failed_rows(output_path_dv, llm_client, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
client = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_dz_common_prompts.csv'
retry_failed_rows(path, client, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_dz_common_prompts.csv'
retry_failed_rows(path, client, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:

path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_hi_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_hi_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_ne_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_ne_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Updated response for row 21
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Updated response for row 44
Processed row 15
Updated response for row 45
Processed row 16
Updated response for row 47
Processed row 17
Updated response for row 50
Processed row 18
Updated response for row 52
Processed row 19
Updated response for row 58
Processed row 20
Updated response for row 60
Processed row 21
Updated response for row 62
Processed row 22
Updated response for row 64
Processed row 23
Updated response for row 75
Processed row 24
Updated response for row 77
Processed row 25
Updated response for row 80
Processed row 26
Updated response for row 82
Processed row 27
Updated response for row 84
Processed row 28
Updated response for row 93
Processed row 29
Updated response for row 96
Processed row 30
Updated

In [ ]:
token = '' # out for llama
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_ps_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
token = '' # out for llama
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_ps_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Updated response for row 221
Processed ro

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_si_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Updated response for row 4
Processed row 0
Updated response for row 9
Processed row 1
Updated response for row 10
Processed row 2
Updated response for row 12
Processed row 3
Updated response for row 14
Processed row 4
Processed row 5
Processed row 6
Updated response for row 17
Processed row 7
Updated response for row 19
Processed row 8
Updated response for row 21
Processed row 9
Updated response for row 26
Processed row 10
Processed row 11
Updated response for row 37
Processed row 12
Updated response for row 39
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Updated response for row 50
Processed row 18
Updated response for row 52
Processed row 19
Updated response for row 58
Processed row 20
Processed row 21
Updated response for row 62
Processed row 22
Updated response for row 64
Processed row 23
Updated response for row 75
Processed row 24
Updated response for row 77
Processed row 25
Updated response for row 80
Processed row 26
Updated response for 

In [ ]:
token = '' # out for llama
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_si_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
token = ''
client2 = LLMClient(token)
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_ta_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_ta_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_ur_common_prompts.csv'
retry_failed_rows(path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Updated response for row 10
Processed row 2
Updated response for row 12
Processed row 3
Updated response for row 14
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Updated response for row 37
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Updated response for row 45
Processed row 16
Processed row 17
Updated response for row 50
Processed row 18
Updated response for row 52
Processed row 19
Updated response for row 58
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Updated response for row 77
Processed row 25
Updated response for row 80
Processed row 26
Updated response for row 82
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Updated response for row 98
Processed row 31
Updated response for row 104
Processed row 32
Processed row 33
Updated response for row 109
Processed row 34
Updated response for 

In [ ]:
path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_ur_common_prompts.csv'
retry_failed_rows(path, client2, 'llama3-70b-8192')

Updated response for row 4
Processed row 0
Updated response for row 9
Processed row 1
Updated response for row 10
Processed row 2
Updated response for row 12
Processed row 3
Updated response for row 14
Processed row 4
Updated response for row 15
Processed row 5
Updated response for row 16
Processed row 6
Updated response for row 17
Processed row 7
Updated response for row 19
Processed row 8
Updated response for row 21
Processed row 9
Updated response for row 26
Processed row 10
Updated response for row 31
Processed row 11
Updated response for row 37
Processed row 12
Updated response for row 39
Processed row 13
Updated response for row 40
Processed row 14
Updated response for row 44
Processed row 15
Updated response for row 45
Processed row 16
Updated response for row 47
Processed row 17
Updated response for row 50
Processed row 18
Updated response for row 52
Processed row 19
Updated response for row 58
Processed row 20
Updated response for row 60
Processed row 21
Updated response for r

In [ ]:
token = ''
client2 = LLMClient(token)
file_path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_bn_common_prompts.csv'
retry_failed_rows(file_path, client2, 'mixtral-8x7b-32768')

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
import os
import csv
token = ''
file_path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/mixed_language_prompts.csv'
response_folder = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response'
process_file_with_model(file_path, response_folder, token, 'mixtral-8x7b-32768')

Processed row 1876 with model mixtral-8x7b-32768
Processed row 31 with model mixtral-8x7b-32768
Processed row 354 with model mixtral-8x7b-32768
Processed row 321 with model mixtral-8x7b-32768
Processed row 1672 with model mixtral-8x7b-32768
Processed row 125 with model mixtral-8x7b-32768
Processed row 1026 with model mixtral-8x7b-32768
Processed row 355 with model mixtral-8x7b-32768
Processed row 1661 with model mixtral-8x7b-32768
Processed row 1444 with model mixtral-8x7b-32768
Processed row 1414 with model mixtral-8x7b-32768
Processed row 1129 with model mixtral-8x7b-32768
Processed row 773 with model mixtral-8x7b-32768
Processed row 1131 with model mixtral-8x7b-32768
Processed row 1454 with model mixtral-8x7b-32768
Processed row 1779 with model mixtral-8x7b-32768
Processed row 1208 with model mixtral-8x7b-32768
Processed row 77 with model mixtral-8x7b-32768
Processed row 9 with model mixtral-8x7b-32768
Processed row 1900 with model mixtral-8x7b-32768
Processed row 308 with model mix

In [ ]:
import os
import csv
token = ''
file_path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/mixed_language_prompts.csv'
response_folder = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response'
process_file_with_model(file_path, response_folder, token, 'llama3-70b-8192')

Processed row 1876 with model llama3-70b-8192
Processed row 31 with model llama3-70b-8192
Processed row 354 with model llama3-70b-8192
Processed row 321 with model llama3-70b-8192
Processed row 1672 with model llama3-70b-8192
Processed row 125 with model llama3-70b-8192
Processed row 1026 with model llama3-70b-8192
Processed row 355 with model llama3-70b-8192
Processed row 1661 with model llama3-70b-8192
Processed row 1444 with model llama3-70b-8192
Processed row 1414 with model llama3-70b-8192
Processed row 1129 with model llama3-70b-8192
Processed row 773 with model llama3-70b-8192
Processed row 1131 with model llama3-70b-8192
Processed row 1454 with model llama3-70b-8192
Processed row 1779 with model llama3-70b-8192
Processed row 1208 with model llama3-70b-8192
Processed row 77 with model llama3-70b-8192
Processed row 9 with model llama3-70b-8192
Processed row 1900 with model llama3-70b-8192
Processed row 308 with model llama3-70b-8192
Processed row 1316 with model llama3-70b-8192
P

In [ ]:
import csv
import os

def process_csv(input_file, output_file, llm_client, model):
    # Open the input CSV file
    with open(input_file, mode="r", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)  # Read all rows into a list

    # Open the output CSV file in append mode
    with open(output_file, mode="a", encoding="utf-8", newline="") as outfile:
        fieldnames = ["id", "original_prompt","response"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the header only if the file is empty
        if outfile.tell() == 0:
            writer.writeheader()

        # Process each row
        for row in rows:
            id = row["id"]
            translated_prompt = row["original_prompt"]

            # Generate the response using the specified model
            response = llm_client.generate_response(translated_prompt, model)

            # Write the result to the output CSV immediately
            writer.writerow({
                "id": id,
                "original_prompt": translated_prompt,
                "response": response,
            })

            print(f"Processed row {id} with model {model}")  # Log progress

    print(f"Responses saved to {output_file}")

In [ ]:
token = ''
client2 = LLMClient(token)
path_in = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/en_common_prompts.csv'
path_out = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/mixtral-8x7b-32768_en_common_prompts.csv'
process_csv(path_in, path_out, client2, 'mixtral-8x7b-32768')

Processed row 4 with model mixtral-8x7b-32768
Processed row 9 with model mixtral-8x7b-32768
Processed row 10 with model mixtral-8x7b-32768
Processed row 12 with model mixtral-8x7b-32768
Processed row 14 with model mixtral-8x7b-32768
Processed row 15 with model mixtral-8x7b-32768
Processed row 16 with model mixtral-8x7b-32768
Processed row 17 with model mixtral-8x7b-32768
Processed row 19 with model mixtral-8x7b-32768
Processed row 21 with model mixtral-8x7b-32768
Processed row 26 with model mixtral-8x7b-32768
Processed row 31 with model mixtral-8x7b-32768
Processed row 37 with model mixtral-8x7b-32768
Processed row 39 with model mixtral-8x7b-32768
Processed row 40 with model mixtral-8x7b-32768
Processed row 44 with model mixtral-8x7b-32768
Processed row 45 with model mixtral-8x7b-32768
Processed row 47 with model mixtral-8x7b-32768
Processed row 50 with model mixtral-8x7b-32768
Processed row 52 with model mixtral-8x7b-32768
Processed row 58 with model mixtral-8x7b-32768
Processed row 6

In [ ]:
token = ''
client2 = LLMClient(token)
path_in = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/en_common_prompts.csv'
path_out = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/Response/llama3-70b-8192_en_common_prompts.csv'
process_csv(path_in, path_out, client2, 'llama3-70b-8192')

Processed row 4 with model llama3-70b-8192
Processed row 9 with model llama3-70b-8192
Processed row 10 with model llama3-70b-8192
Processed row 12 with model llama3-70b-8192
Processed row 14 with model llama3-70b-8192
Processed row 15 with model llama3-70b-8192
Processed row 16 with model llama3-70b-8192
Processed row 17 with model llama3-70b-8192
Processed row 19 with model llama3-70b-8192
Processed row 21 with model llama3-70b-8192
Processed row 26 with model llama3-70b-8192
Processed row 31 with model llama3-70b-8192
Processed row 37 with model llama3-70b-8192
Processed row 39 with model llama3-70b-8192
Processed row 40 with model llama3-70b-8192
Processed row 44 with model llama3-70b-8192
Processed row 45 with model llama3-70b-8192
Processed row 47 with model llama3-70b-8192
Processed row 50 with model llama3-70b-8192
Processed row 52 with model llama3-70b-8192
Processed row 58 with model llama3-70b-8192
Processed row 60 with model llama3-70b-8192
Processed row 62 with model llama3